In [ ]:
%matplotlib inline

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from xnoah.data_matrix import stack_cat, unstack_cat
import holoviews as hv
hv.extension('bokeh')

In [ ]:
data = xr.open_dataset("../data/processed/regridded.nc", chunks={'t':100})

In [ ]:
data

Plot basic time and horizontally averaged fields

In [ ]:
mu = xr.open_dataset("../data/stats/mu.nc")
sig = xr.open_dataset("../data/stats/sig.nc")

In [ ]:
fig, axs = plt.subplots(1,3, sharey=True)

for i, k in enumerate(sig.data_vars):
    axs[i].set_title(k)
    axs[i].plot(sig[k], sig.z, c='b')
    axs[i].plot(mu[k], mu.z, c='r')
    

axs[0].set_ylabel('z')   

Take SVD of the data

In [ ]:
nsvd = 2000
rand = np.random.choice(len(data.t), nsvd)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
X = stack_cat(data['uY'].to_dataset().isel(t=rand), "features", ["x", "z"])


mod = PCA(n_components=10).fit(X)

Plot the principle components. the time series

In [ ]:
Xn = data['uY'].stack(features=['x', 'z'])

pcs = mod.transform(Xn)

In [ ]:
plt.plot(pcs[:,5])

In [ ]:
plt.plot(mod.explained_variance_ratio_)

what about with just SVD with vertical profiles

In [ ]:
stacked = data['uY'].stack(samples=["x", "t"], features=["z"])

In [ ]:
nsvd = 2000
rand_inds = np.random.choice(len(stacked.samples), nsvd)

In [ ]:
X = stacked[rand_inds]

In [ ]:
pca_vert = PCA(n_components=10).fit(X)

In [ ]:
eof = hv.Dataset(( np.arange(1,11), data.z.values , pca_vert.components_.T),
                 kdims=['m', 'z'], vdims=[' '])

In [ ]:
%%opts Curve[invert_axes=True width=150, height=200]

hv.NdLayout(eof.to(hv.Curve, 'z')).cols(5)

These are probably just the solutions to the sturm liouville problem associated with the mean temperature stratification. Whether this is interesting or not is a matter of your persepctive. From my perspective, this just says that the modes of the data are the same as what you get by some kind of random sampling of the equilibrium distribution. 


Also this method throws out the importance of neighboring grid cells, which should definitely be included.
